<a href="https://colab.research.google.com/github/ro-okie/VGG-16_NET/blob/main/VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# defining relevant libraries
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device.type)

cuda


In [ ]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False
                ):
  
  normalize = transforms.Normalize(
      mean = [0.4914, 0.4822, 0.4465],
      std =[0.2023, 0.1994, 0.2010],
      )
  
  transform  = transforms.Compose([
      transforms.Resize((227,227)),
      transforms.ToTensor(),
      normalize
  ])

  if test:
    test_dataset = datasets.CIFAR100(
        root=data_dir, train=False,
        transform=transform,download=True
    )

    test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=shuffle)

    return test_loader

  else:
    train_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        transform=transform,download=True
    )

    valid_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        transform=transform,download=True
    )

    num_train  = len(train_dataset)
    indices = list(range(num_train))

    if shuffle:
      np.random.seed(random_seed)
      np.random.shuffle(indices)


    split = int(np.floor(valid_size*num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, sampler = train_sampler)
    valid_loader = DataLoader(dataset = valid_dataset, batch_size = batch_size, sampler = valid_sampler)


    return (train_loader, valid_loader)


train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class VGG_16(nn.Module):
  def __init__(self, num_classes):
    super().__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )

    self.layer4 = nn.Sequential(
        nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    self.layer5 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
      
    )

    self.layer6 = nn.Sequential(
        nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(256),
        nn.ReLU()
    )

    self.layer7 = nn.Sequential(
        nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    self.layer8 = nn.Sequential(
        nn.Conv2d(256, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
      
    )

    self.layer9 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )

    self.layer10 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    self.layer11 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
      
    )

    self.layer12 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )

    self.layer13 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    self.fc1 = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(7*7*512, 4096),
        nn.ReLU()
    )
    self.fc2 = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU()
    )
    self.fc3 = nn.Sequential(
        nn.Linear(4096, num_classes)
    )


  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = self.layer6(out)
    out = self.layer7(out)
    out = self.layer8(out)
    out = self.layer9(out)
    out = self.layer10(out)
    out = self.layer11(out)
    out = self.layer12(out)
    out = self.layer13(out)

    out = out.reshape(out.size(0), -1)

    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
    return out






In [ ]:
# defining relevant variables
batch_size = 64
num_classes = 100
learning_rate = 0.005
num_epochs = 20

In [ ]:
# defining hyperparmeters

model = VGG_16(num_classes)
model.cuda()

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay = 0.005, momentum = 0.9)

total_steps = len(train_loader)

In [ ]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    
    output = model(images)

    loss = loss_fn(output, labels)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

  print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_steps, loss.item()))
  
  # validation

  with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in valid_loader:
      images = images.to(device)
      labels = labels.to(device)

      output = model(images)

      _, predicted = torch.max(output.data, 1)

      total += labels.size(0)
      correct += (predicted==labels).sum().item()

      del images, labels, output
  print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))     
    

Epoch [1/20], Step [704/704], Loss: 3.7204
Accuracy of the network on the 5000 validation images: 13.9 %


KeyboardInterrupt: ignored

In [ ]:
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)

      output = model(images)

      _, predicted = torch.max(output.data, 1)

      total += labels.size(0)
      correct += (predicted==labels).sum().item()

      del images, labels, output
    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))     